In [65]:
import pymongo
from dotenv import load_dotenv
import os
import itertools
from math import pi, sqrt, cos

load_dotenv()


True

In [66]:

mongodb_uri = os.environ.get("MONGODB_URI")
client = pymongo.MongoClient(mongodb_uri)
db = client[os.environ.get("MONGODB_DB")]

classes = db["classes"]
locations = db["locations"]


In [67]:

weights = {
    "distance": 25,
    "back_to_back": 15,
    "time": 50,
    "section": 10,
    "lunch": 7.5
}

class ScheduleException(Exception):
    pass

class ScheduleOverlapException(Exception):
    # When a schedule is not possible because 2 classes overlap time-wise
    pass

In [68]:

def check_for_no_section(classes_to_take, open_sections_filter_applied):
    for clas in classes_to_take:
        if len(clas["sections"]) == 0:
            if (open_sections_filter_applied):
                raise ScheduleException(f"No open sections found for {clas['code']} {clas['number']} in your specified time range. Please try again with a different time range.")
            else:
                raise ScheduleException(f"No sections found for {clas['code']} {clas['number']} in your specified time range. Please try again with a different time range.")


def fetch_section_data(class_list):
    """
    Fetches the section data from the database
    """
    for class_ in class_list:
        clas_mongo_object = classes.find_one({"code": class_["code"], "number": class_[
                                             "number"], "year": 2024, "semester": "spring"})
        class_["sections"] = []
        for section in clas_mongo_object["sections"]:
            if section["crn"] in class_["crn_list"]:

                if section["type_code"] == "OLC":
                    continue

                section["class"] = {
                    "code": class_["code"],
                    "number": class_["number"]
                }
                
                class_["sections"].append(section)

    return class_list


def is_preferred_time(section, start_time, end_time):
    earliest_meeting = start_time
    latest_meeting = end_time

    if section['start_time'] is None or section['end_time'] is None:
        return True
    if section['start_time'].hour < earliest_meeting:
        earliest_meeting = section['start_time'].hour
    if section['end_time'].hour > latest_meeting:
        latest_meeting = section['end_time'].hour

    return earliest_meeting >= start_time and latest_meeting <= end_time


def delete_unpreferred_sections(class_list, start_time, end_time):
    for clas in class_list:
        clas["sections"] = [section for section in clas['sections'] if is_preferred_time(section, start_time, end_time)]
    return class_list


def delete_closed_sections(class_list, open_sections_only):
    if open_sections_only:
        for clas in class_list:
            clas["sections"] = [section for section in clas['sections'] if section['enrollment_status'].lower() != 'closed']
    return class_list


def apply_hard_filters(user_preferences):
    class_list = fetch_section_data(user_preferences["classes"])
    class_list = delete_unpreferred_sections(class_list, user_preferences["start_time"], user_preferences["end_time"])
    check_for_no_section(class_list, False)  # Check if any class has no sections

    class_list = delete_closed_sections(class_list, user_preferences["open_sections_only"])

    check_for_no_section(class_list, True)  # Check if any class has no sections

    return class_list


def remove_duplicates(grouped_sections, pref_sections):
    hash_list = set()
    new_list = []
    for section in grouped_sections:
        section_hash = f"{section['coordinates']}_{section['start_time']}_{section['end_time']}_{section['days']}"
        if section_hash not in hash_list:
            hash_list.add(section_hash)
            new_list.append(section)
        elif section['crn'] in pref_sections:
            new_list.append(section)

    return new_list


In [69]:


def generate_schedule_combinations(class_list, user_preferences):
    """
    Generates all possible schedules
    """
    groups = {}

    # Split classes into groups based on meeting types
    for clas in class_list:
        base_group_name = f"{clas['code']}_{clas['number']}_"
        for section in clas['sections']:
            group_key = base_group_name + "_" + section['type_code']
            groups[group_key] = groups.get(group_key, []) + [section]

    un_duplicated_groups = {}

    # Remove groups with section which have the same days, times, and locations
    for key, value in groups.items():
        un_duplicated_groups[key] = remove_duplicates(value, user_preferences["pref_sections"])

    possible_schedules = itertools.product(*un_duplicated_groups.values())

    return possible_schedules

In [70]:


def convert_to_time_based(list_of_sections):
    schedule_slots = {
        'W': [None]*23,
        'R': [None]*23,
        'F': [None]*23,
        'T': [None]*23,
        'M': [None]*23,
    }

    for section in list_of_sections:
        if section['start_time'] is None or section['end_time'] is None:
            continue

        start_hour = section['start_time'].hour
        end_hour = section['end_time'].hour

        for day in section['days'].strip():
            for hour in range(start_hour, end_hour+1):
                if schedule_slots[day][hour] is not None:
                    raise ScheduleOverlapException(f"Schedule overlap detected for {section['class']['code']} {section['class']['number']} and {schedule_slots[day][hour]['class']['code']} {schedule_slots[day][hour]['class']['number']}")
                schedule_slots[day][hour] = section

    return schedule_slots



In [71]:
def pythagorian_distance(coords1, coords2):
    lat1 = coords1[0]  
    lon1 = coords1[1]  
    lat2 = coords2[0] 
    lon2 = coords1[1] 

    avgLatDeg = (lat1 + lat2) / 2
    avgLat = avgLatDeg * (pi/180)

    d_ew = (lon2 - lon1) * cos(avgLat)
    d_ns = (lat2 - lat1)
    approxDegreesSq = (d_ew * d_ew) + (d_ns * d_ns)
    d_degrees = sqrt(approxDegreesSq)
    EarthAvgMeterPerGreatCircleDegree = (6371000 * 2 * pi) / 360
    return d_degrees * EarthAvgMeterPerGreatCircleDegree

def list_of_successive_distances(schedule):

    list_of_distances = []
    for day in schedule:
        for hour in range(0, 23):
            if schedule[day][hour] is not None and schedule[day][hour + 1] is not None:
                coords1 = schedule[day][hour]['coordinates']
                coords2 = schedule[day][hour + 1]['coordinates']
                if coords1 is not None and coords2 is not None:
                    list_of_distances.append(pythagorian_distance(coords1, coords2))

    return [x for x in list_of_distances if x != 0.0]


def distance_score(list_of_sections, max_distance) -> float:
    # Returns a score between -distance_weight and distance_weight based on how far apart the classes are
    list_of_distances = list_of_successive_distances(list_of_sections)

    if len(list_of_distances) == 0:
        return 0

    mean_distance = sum(list_of_distances) / len(list_of_distances)

    # Find the distance score
    if mean_distance > max_distance:
        percent_over = (mean_distance - max_distance) / max_distance
        return -(percent_over * weights["distance"])
    elif mean_distance < max_distance:
        percent_under = (max_distance - mean_distance) / max_distance
        return percent_under * weights["distance"]
    else:
        return 0



In [72]:

def back_to_back_score(list_of_sections, back_to_back) -> float:
    back_to_back_count = 0
    total_count = 0

    for day in list_of_sections:
        for hour in range(0, 23):
            if list_of_sections[day][hour] is not None and list_of_sections[day][hour + 1] is not None:
                total_count += 1
                if list_of_sections[day][hour]['crn'] != list_of_sections[day][hour + 1]['crn']:
                    back_to_back_count += 1

    if total_count == 0:
        return 0
    elif back_to_back:
        return (back_to_back_count / total_count) * weights["back_to_back"]
    else:
        return (-back_to_back_count / total_count) * weights["back_to_back"]



In [73]:

def time_score(list_of_sections, pref_time):
    time_list = []

    for day in list_of_sections:
        for hour in range(0, 23):
            if list_of_sections[day][hour] is not None:
                time_list.append(hour)

    if len(time_list) == 0:
        return 0
    else:
        mean_time = sum(time_list) / len(time_list)
        return (1 - (abs(mean_time - pref_time) / pref_time)) * weights["time"]



In [74]:

def section_score(list_of_sections, pref_sections):
    section_count = 0
    for day in list_of_sections:
        for hour in range(0, 23):
            if list_of_sections[day][hour] is not None and list_of_sections[day][hour]['crn'] in pref_sections:
                section_count += 1

    return section_count * weights["section"]



In [75]:

def lunch_score(list_of_sections, lunch):
    lunch_count = 0
    for day in list_of_sections:
        for hour in range(0, 23):
            if list_of_sections[day][hour] is None and hour >= lunch['start'] and hour < lunch['end']:
                duration_hours = [list_of_sections[day][x] is None for x in range(hour, hour + lunch['duration']) if x < 23]
                if all(duration_hours):
                    lunch_count += 1
                    break
    return (lunch_count - 5) * weights["lunch"]


In [76]:

def compute_schedule_score(list_of_sections, user_preferences) -> float:
    schedule = convert_to_time_based(list_of_sections)
    
    if user_preferences['max_distance'] > 10_000:
        ds = 0
    else :
        ds = distance_score(schedule, user_preferences["max_distance"])

    btb = back_to_back_score(schedule, user_preferences["back_to_back"])
    ts = time_score(schedule, user_preferences["pref_time"])
    ss = section_score(schedule, user_preferences["pref_sections"])
    ls = lunch_score(schedule, user_preferences["lunch"])
    return (ds + btb + ts + ss + ls), schedule


In [77]:

def sort_schedules(possible_schedules, user_preferences):
    sorted_schedules = []
    for schedule in possible_schedules:
        try :
            score, time_schedule = compute_schedule_score(schedule, user_preferences)
            sorted_schedules.append(
                {
                    "time_schedule": time_schedule,
                    "schedule": schedule,
                    "score": score
                }
            )
        except ScheduleOverlapException:
            continue
    if len(sorted_schedules) == 0:
        raise ScheduleException("No possible schedules found. Please try again with different preferences.")
    sorted_schedules.sort(key=lambda x: x["score"], reverse=True)
    print(f"Number of possible schedules: {len(sorted_schedules)}")
    return sorted_schedules



In [78]:

def get_schedule(user_preferences):
    """
    Returns the schedule for the user
    """

    class_list = apply_hard_filters(user_preferences)
    print('Applied hard filters')
    possible_schedules = generate_schedule_combinations(class_list, user_preferences)
    print('Generated schedule combinations')
    sorted_schedules = sort_schedules(possible_schedules, user_preferences)
    print('Sorted schedules')
    return sorted_schedules[:5]



In [1]:
import json
user_prefs = {"classes": [{"code": "ECE", "number": "110", "crn_list": ["32463", "32460", "52912", "32470", "52914", "52910", "32466", "32461", "52913", "32456", "52911", "63640", "32459", "32464", "32471", "52909", "61723", "57693"]}, {"code": "ECE", "number": "220", "crn_list": ["61635", "61638", "61640", "61641", "61642", "61643", "64369", "61629", "63649", "63650"]}, {"code": "PHYS", "number": "212", "crn_list": ["38029", "38032", "57914", "56034", "58663", "51211", "58664", "53105", "38051", "38057", "38104", "38059", "58662", "57937", "38062",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                        "57963", "58985", "58987", "58988", "58978", "58979"]}, {"code": "MATH", "number": "241", "crn_list": ["50318", "46053", "46054", "46056", "46057", "46058", "48355", "46061", "46062", "46065", "46066", "46063", "46064", "59561", "58071", "46060", "46067", "46059", "52979", "46070", "46071", "46072", "46073", "47543", "67808", "50322", "55923", "55924", "55925", "55922", "55926", "56027", "56026", "55921", "46074"]}, {"code": "CWL", "number": "207", "crn_list": ["65011"]}], "pref_sections": ["52914", "63640", "32471", "61643", "61642", "38096", "38088", "50318"], "classes_1": ["ECE 110", "ECE 220", "PHYS 212", "MATH 241", "CWL 207"], "open_sections_only": False, "start_time": 10, "end_time": 21, "pref_time": 16, "lunch": {"start": 11, "end": 14, "duration": 1}, "max_distance": 800, "back_to_back": True}
# test_outputs = get_schedule(user_prefs)
# print(json.dumps(test_outputs, indent=4, sort_keys=True, default=str))

print(json.dumps(user_prefs, indent=4, default=str))

{
    "classes": [
        {
            "code": "ECE",
            "number": "110",
            "crn_list": [
                "32463",
                "32460",
                "52912",
                "32470",
                "52914",
                "52910",
                "32466",
                "32461",
                "52913",
                "32456",
                "52911",
                "63640",
                "32459",
                "32464",
                "32471",
                "52909",
                "61723",
                "57693"
            ]
        },
        {
            "code": "ECE",
            "number": "220",
            "crn_list": [
                "61635",
                "61638",
                "61640",
                "61641",
                "61642",
                "61643",
                "64369",
                "61629",
                "63649",
                "63650"
            ]
        },
        {
            "code": "PHYS",
           